In [36]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [37]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [38]:

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']


In [39]:

def load_data( label_name ='Species'):
    """Parses the csv file in TRAIN_URL and TEST_URL."""
    
    train_path = tf.keras.utils.get_file(fname=TRAIN_URL.split('/')[-1],
                                         origin=TRAIN_URL)
    train = pd.read_csv(filepath_or_buffer=train_path,names=CSV_COLUMN_NAMES,header=0)
    
    train_features, train_label = train, train.pop(label_name)
   
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)
    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_features, test_label = test, test.pop(label_name)

    
    return (train_features, train_label), (test_features, test_label)

In [40]:
(train_features, train_label), (test_features, test_label) = load_data()

In [41]:
train_label=np.array(train_label)
train_label
test_label=np.array(test_label)
test_label

array([1, 2, 0, 1, 1, 1, 0, 2, 1, 2, 2, 0, 2, 1, 1, 0, 1, 0, 0, 2, 0, 1,
       2, 1, 1, 1, 0, 1, 2, 1])

In [42]:
train_features_dict={}
for col in train_features.columns:
    train_features_dict[col]=list(train_features[col].values)
test_features_dict={}
for col in test_features.columns:
    test_features_dict[col]=list(test_features[col].values)

In [43]:
my_feature_columns = [
    tf.feature_column.numeric_column(key='SepalLength'),
    tf.feature_column.numeric_column(key='SepalWidth'),
    tf.feature_column.numeric_column(key='PetalLength'),
    tf.feature_column.numeric_column(key='PetalWidth')
]

In [44]:
classifier=tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        hidden_units=[10, 10],
        n_classes=3)
hidden_units=[10,10]

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpp0sof_um', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f28edc77ef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [45]:
def train_input_fn(features, labels, batch_size):
        dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
        dataset = dataset.shuffle(buffer_size=1000).repeat(count=None).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

In [46]:
classifier.train(
        input_fn=lambda:train_input_fn(train_features_dict, train_label, 100),
        steps= 1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpp0sof_um/model.ckpt.
INFO:tensorflow:loss = 194.55972, step = 1
INFO:tensorflow:global_step/sec: 438.077
INFO:tensorflow:loss = 21.73808, step = 101 (0.229 sec)
INFO:tensorflow:global_step/sec: 693.495
INFO:tensorflow:loss = 10.112274, step = 201 (0.144 sec)
INFO:tensorflow:global_step/sec: 643.666
INFO:tensorflow:loss = 10.233298, step = 301 (0.155 sec)
INFO:tensorflow:global_step/sec: 622.576
INFO:tensorflow:loss = 10.679776, step = 401 (0.161 sec)
INFO:tensorflow:global_step/sec: 662.649
INFO:tensorflow:loss = 4.8249836, step = 501 (0.151 sec)
INFO:tensorflow:global_step/sec: 669.421
INFO:tensorflow:loss = 6.5177927, step = 601 (0.149 sec)
INFO:tensorflow:global_step/sec: 633.297
INFO:tensorflow:l

In [47]:

eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_features_dict, test_label, 100))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


def eval_input_fn(features, labels=None, batch_size=None):
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert inputs to a tf.dataset object.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-06-07:10:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpp0sof_um/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-06-07:10:34
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, average_loss = 0.065355636, global_step = 1000, loss = 1.9606692
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpp0sof_um/model.ckpt-1000

Test set accuracy: 0.967



In [71]:
predict_x = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth': [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth': [0.5, 1.5, 2.1],
    }
    
predictions=classifier.predict(
 input_fn=lambda:eval_input_fn(predict_x, batch_size=100))
for pred_dict, expec in zip(predictions, train_label):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    print(template.format(train_label[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpp0sof_um/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "2" (99.7%), expected "2"

Prediction is "1" (99.6%), expected "1"

Prediction is "2" (97.5%), expected "2"
